In [2]:
%pip install selenium webdriver

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement webdriver (from versions: none)
ERROR: No matching distribution found for webdriver


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

"""li.sports_list 내부의 a 태그 정보 추출"""
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = None
try:
    # 드라이버 설정
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    # 페이지 접속
    url = "https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?idx=baseball"
    driver.get(url)
    logger.info("페이지 로드 완료")

    # 데이터 추출
    stadium_link_dic = {}
    sports_list_items = driver.find_elements(By.CSS_SELECTOR, 'li.sports_list')
    logger.info(f"총 {len(sports_list_items)}개의 sports_list 항목 발견")

    for li_item in sports_list_items:
        try:
            # 각 li 내부의 span들에서 a 태그 찾기
            spans = li_item.find_elements(By.CSS_SELECTOR, 'span')
            for span in spans:
                try:
                    a_tag = span.find_element(By.CSS_SELECTOR, "a[href*='PouVSwRTRIN&idx=baseball']")
                    href = a_tag.get_attribute('href')
                    
                    try:
                        name = a_tag.text
                    except NoSuchElementException:
                        name = ""
                    stadium_link_dic[name] = href

                    logger.debug(f"추출 완료: {name} - {href}")
                    
                except NoSuchElementException:
                    continue


        except Exception as e:
            logger.warning(f"항목 처리 중 오류: {str(e)}")
            continue

except Exception as e:
    logger.error(f"전체 프로세스 오류: {str(e)}")
finally:
    if driver:
        driver.quit()
        logger.info("드라이버 종료 완료")


print("\n[추출 결과]")
print(stadium_link_dic)

        

ModuleNotFoundError: No module named 'webdriver_manager'

In [21]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

def scrape_stadium_forecast(stadium_link_dic, stadium_name):
    """
    경기장 이름을 입력받아 해당 페이지의 날씨 정보 추출
    :param stadium_link_dic: {'경기장명': '링크'} 형식의 딕셔너리
    :param stadium_name: 조회할 경기장 이름
    :return: 테이블 헤더와 데이터를 포함한 딕셔너리
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = None

    try:
        
        # 드라이버 설정
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=chrome_options
        )
        
        # 해당 경기장 페이지 접속
        driver.get(stadium_link_dic[stadium_name])
        logger.info(f"{stadium_name} 페이지 로드 시작")

        # 테이블 로딩 대기 (최대 15초)
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'li.lifestyle_present_forecast_table'))
            )
        except TimeoutException:
            logger.error("테이블 요소를 찾을 수 없음")
            return None

        # 테이블 데이터 추출
        table = driver.find_element(By.CSS_SELECTOR, 'li.lifestyle_present_forecast_table')
        
        # thead에서 컬럼명 추출
        headers = [th.text for th in table.find_elements(By.CSS_SELECTOR, 'thead th')]
        
        # tbody에서 데이터 추출
        rows = []
        for tr in table.find_elements(By.CSS_SELECTOR, 'tbody tr'):
            row_data = [td.text for td in tr.find_elements(By.TAG_NAME, 'td')]
            if row_data:  # 빈 행 제외
                rows.append(row_data)

        # 결과 구조화
        result = {
            'stadium': stadium_name,
            'headers': headers,
            'data': rows,
            # 'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        return result

    except Exception as e:
        logger.error(f"에러 발생: {str(e)}")
        return None
    finally:
        if driver:
            driver.quit()

In [22]:
scrape_stadium_forecast(stadium_link_dic, "잠실야구장")

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\user\.wdm\drivers\chromedriver\win64\135.0.7049.95\chromedriver-win32/chromedriver.exe] found in cache
INFO:__main__:잠실야구장 페이지 로드 시작


{'stadium': '잠실야구장',
 'headers': ['기온', '체감', '풍향', '풍속', '강수', '습도'],
 'data': [['21.6℃', '21.6℃', '남남서', '3.6m/s', '-mm', '38%']]}